In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('data/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df = df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [6]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
X=df.drop(columns=['Survived'])
y=df['Survived']

In [8]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [9]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=45)

In [11]:
X_train.shape

(712, 7)

In [12]:
X_test.shape

(179, 7)

In [13]:
X_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 332 to 414
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       568 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


In [15]:

# Identify categorical and numerical columns
numerical_features = ['Age', 'Fare','Parch','SibSp']
categorical_features = ['Sex', 'Embarked', 'Pclass']

# Define preprocessing steps for numerical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define preprocessing steps for categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [16]:
# Model Selection
trf5 =DecisionTreeClassifier()

In [17]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', trf5)])


In [18]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare', 'Parch',
                                                   'SibSp']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Pclass'])])),
                ('classifier', DecisionTreeClassifier())])

In [19]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare', 'Parch',
                                                   'SibSp']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Pclass'])])),
                ('classifier', DecisionTreeClassifier())])

In [20]:
y_pred=pipeline.predict(X_test)

In [21]:
y_pred

array([0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0])

In [22]:
pd.DataFrame({'y_test':y_test,'y_predict':y_pred})

,y_test,y_predict
248,1,0
197,0,0
133,1,1
169,0,1
736,0,0
...,...,...
886,0,0
751,1,1
804,1,0
596,1,1


In [27]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score
print("Accuracy",accuracy_score(y_test,y_pred))
print("Precision Score",precision_score(y_test,y_pred))

Accuracy 0.776536312849162
Precision Score 0.6341463414634146


In [26]:
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))

Confusion Matrix:
 [[87 30]
 [10 52]]
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.74      0.81       117
           1       0.63      0.84      0.72        62

    accuracy                           0.78       179
   macro avg       0.77      0.79      0.77       179
weighted avg       0.81      0.78      0.78       179



In [36]:
## Cross Validation using Pipe Line
from sklearn.model_selection import cross_val_score
scores  = cross_val_score(pipeline,X_train,y_train,cv=10,scoring='accuracy')
for i in range(len(scores)):
    print("Fold",i+1,":",scores[i])
print("Maximum Accuracy: ", max(scores))

Fold 1 : 0.8055555555555556
Fold 2 : 0.7361111111111112
Fold 3 : 0.7887323943661971
Fold 4 : 0.7887323943661971
Fold 5 : 0.7464788732394366
Fold 6 : 0.7746478873239436
Fold 7 : 0.7464788732394366
Fold 8 : 0.7464788732394366
Fold 9 : 0.7464788732394366
Fold 10 : 0.7183098591549296
Maximum Accuracy:  0.8055555555555556


In [41]:
## Cross Validation using Pipe Line
from sklearn.model_selection import cross_val_score
scores  = cross_val_score(pipeline,X_test,y_pred,cv=10,scoring='accuracy')
for i in range(len(scores)):
    print("Fold",i+1,":",scores[i])
print("Maximum Accuracy: ", max(scores))

Fold 1 : 0.7222222222222222
Fold 2 : 0.6666666666666666
Fold 3 : 0.8888888888888888
Fold 4 : 0.7777777777777778
Fold 5 : 0.7222222222222222
Fold 6 : 0.7222222222222222
Fold 7 : 0.8888888888888888
Fold 8 : 0.6111111111111112
Fold 9 : 0.7222222222222222
Fold 10 : 0.47058823529411764
Maximum Accuracy:  0.8888888888888888


In [55]:
params = {
    'classifier__max_depth': [1, 2, 3, 4, 5, None],
    'classifier__criterion':['gini','entropy']
}

In [57]:
from sklearn.model_selection import GridSearchCV

# Initialize GridSearchCV
grid = GridSearchCV(pipeline, param_grid=params, cv=5, scoring='accuracy')

# Fit on training data
grid.fit(X_train, y_train)

# Best parameters and accuracy
print("Best Parameters:", grid.best_params_)
print("Best Accuracy:", grid.best_score_)

Best Parameters: {'classifier__criterion': 'gini', 'classifier__max_depth': 3}
Best Accuracy: 0.8117206736925047


## Exporting Model in Pickle

In [58]:
import pickle

In [59]:
pickle.dump(pipeline,open('pipe.pkl','wb'))